# Исследование надежности заемщиков.

Вводные данные - обезличенные сведения о заемщиках в разрезе кредитных договоров и информация о своевременности погашения таких договоров.
Заказчик - кредитный отдел банка.
Задание - определить наличие взаимосвязи семейного положения, уровня дохода, цели кредита на своевременность погашения.
Будут проведены следующие работы по обработке данных:

1. [Обзор данных]
2.1 [Заполнение пропусков]
2.2 [Проверка данных на аномалии]
2.3 [Изменение типов данных]
2.4 [Удаление дубликатов]
2.5 [Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма]
2.6-7 [Категоризация данных]

По результатам предобработки данных будет проведен анализ представленных заказчиком гипотез

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
data_set = pd.read_csv('/datasets/data.csv')
#Посмотрим статистику 
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
print(data_set.isna().sum())#Пропуски встречаются в двух колонках, посчитаем количество строк в которых пропущено хотя бы одно из значений
data_set[(pd.isnull(data_set['total_income'])==True)|(pd.isnull(data_set['total_income'])==True)].count()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Исполнение второго условия показывает, что в строках в которых не заполнено значение "Стаж работы"(days_employed) также не заполнено и значение "совокупный доход"(total_income). Это подтверждается тем фактом что количество строк в которых хотя бы одно из двух данны занчений пропущено, равно количеству строк в которых пропущены оба.

In [3]:
print('Пропущено значений в total_income',data_set['total_income'].isna().sum())
print('Пропущено значений в days_employed',data_set['days_employed'].isna().sum())
print('Строк в которых пропущено хоотя бы одно из указанных значений', len(data_set[(pd.isnull(data_set['total_income'])==True)|(pd.isnull(data_set['total_income'])==True)]))
print('Доля строк с проущенным значением','{:0%}'.format(data_set['total_income'].isna().sum()/len(data_set)))

Пропущено значений в total_income 2174
Пропущено значений в days_employed 2174
Строк в которых пропущено хоотя бы одно из указанных значений 2174
Доля строк с проущенным значением 10.099884%


Очевидно что по какой то причине всегда не заполнялись оба значения - возможно это связано со спецификой отдельного продуктового решения или категории заемщиков. Проверим нет ли очевидных сходств в таких строках.

In [4]:
#Выведем строки в которых есть значения Null, возможно в них есть очевидные сходства:
data_set[pd.isnull(data_set['days_employed'])==True].head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Вывод по обзору данных: 
1. К анализу предтавлены данные о 21525 клиентах. В исходных данных имеются пропуски значений. Доля строк с пропусками 10%.
2. Пропуски встречаются одновременно в столбцах "стаж работы" и "совокупный доход" (если пропущено одно значение, то в этой строке пропущено и другое), это может быть связано со спецификой канала продаж из которого поступал клиент. Возможно при продаже в рамках такого канала продаж эта информация не запрашивалась или не фиксировалась. 
3. Очевидного сходства в строках с пропущенными значениями нет. Есть смысл заполнить эти пропуски медианными значениями.
4. Использование среднего значения затруднено крайними, заведомо ошибочными значениями, в частности далее в п. 2.2.2 будет отражена ситуация с наличием в исходных данных значений трудового стажа в 900 и более лет.

### Шаг 2.1 Заполнение пропусков

In [5]:
#Заполним ранее выявленные систематические пропуски в столбцах days_employed и total_income
data_set['total_income'] = data_set['total_income'].fillna(data_set.groupby('income_type')['total_income'].transform('median'))
data_set['days_employed'] = data_set['days_employed'].fillna(data_set.groupby('income_type')['days_employed'].transform('median'))
print('Медианы дохода по видам занятости', data_set.groupby('income_type')['total_income'].median())
print()
print('Медианы стажа по видам занятости', data_set.groupby('income_type')['total_income'].median())
print()
#Посмотрим статистику 
data_set.info()

Медианы дохода по видам занятости income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Медианы стажа по видам занятости income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-nul

Нулевые строки устранены.

### Шаг 2.2 Проверка данных на аномалии и исправления.

<b>Проверим значения по столбцам на аномалии:

<b>2.2.1. Количество детей (children)
    

In [6]:
# посчитаем распределение заемщиков по значению количества детей
data_set['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Очевидно, что "20" и "-1" это ошибки ввода, которые следует заменить на "2" и "1" соответственно.
Учитвая незначительное количество итераций, заменим конкретные значения через индексацию .loc

In [7]:
data_set.loc[data_set['children']==20,'children']=2
data_set.loc[data_set['children']==-1,'children']=1
data_set['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Некорректные значения по количеству детей устранены

<b> 2.2.2. days_employed (стаж работы)

In [8]:
#Выведем стаж в годах (для удобства) и посмотрим какие встречаются отклонения помимо пропусков:
(data_set[pd.isnull(data_set['days_employed'])==False]['days_employed']//365).sort_values()

16335     -51.0
4299      -49.0
7329      -46.0
17838     -45.0
16825     -45.0
          ...  
4949     1100.0
10991    1100.0
5716     1100.0
10484    1100.0
13420    1100.0
Name: days_employed, Length: 21525, dtype: float64

In [9]:
#Поделим количество "дней стажа" на 3650 или 10 лет целочисленно.
#Так мы получим группировку клиентов по стажу в декадах - групп станет меньше и оценить картину можно будет зрительно

(data_set['days_employed']//3650).value_counts().sort_index()

-6.0          1
-5.0         16
-4.0        161
-3.0        579
-2.0       2392
-1.0      14518
 90.0       147
 91.0       187
 92.0       174
 93.0       193
 94.0       164
 95.0       180
 96.0       156
 97.0       183
 98.0       170
 99.0       162
 100.0      589
 101.0      186
 102.0      203
 103.0      149
 104.0      154
 105.0      170
 106.0      157
 107.0      164
 108.0      194
 109.0      161
 110.0       15
Name: days_employed, dtype: int64

"нормальные" значения стажа хранится в исходной таблице только со знаком "-"
Со знаком + значения от 900 лет стажа и более, что очевидно явяется некорректными данными.
Основной объем значений приходится на "нормальный" стаж:
менее 10 лет 14931 клиентов (60%), 10-20 лет - 2392 клиентов (10%)
все "выбросы" значений по стажу это значения от 91 декады до 111 декад (т.е. от 900 лет до 1100 лет)


In [10]:
#посчитаем какие виды занятости встречаются в выбросах значений по days_employed, установив фильтр в decades_employed
data_set[data_set['days_employed']>0]['income_type'].value_counts()

пенсионер      3856
безработный       2
Name: income_type, dtype: int64

In [11]:
#при этом всего пенсионеров и безработных
data_set['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Стаж некорректно заведен по всем безработным и 90% пенсионеров. 
Возможно по этим категориям клиентов работники банка заводили какое-то "пустое" или "дежурное" значение, т.к. этого требовала программа, либо по таким случаям происходило искажение сведений при сохранении в общую таблицу.
Заменим данные о стаже по таким клиентам на медианное значение

In [12]:
data_set.loc[data_set['days_employed']>0,'days_employed']=(data_set[(data_set['days_employed']<0) & (data_set['income_type']=='пенсионер')]['days_employed'].median())
(data_set['days_employed']//3650).value_counts().sort_index()


-6.0        1
-5.0       16
-4.0      161
-3.0      579
-2.0     2392
-1.0    14518
Name: days_employed, dtype: int64

<b>2.2.3. dob_years (возраст)

In [13]:
data_set['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

#вызывает сомнение возраст 0. Его следует заменить. Ввиду отсутствия крайних значений. 
#Можно заменить на средний возраст тех клиентов по которым возраст проставлен. Поскольку возраст по всем клиентам заведен как целое число, 
#то округлим также и среднее значение, которое мы будем использовать - так сохранится возможность работать с возрастом как с int числом

In [14]:
data_set.loc[data_set['dob_years']==0,'dob_years'] = int(data_set[data_set['dob_years']>0]['dob_years'].mean())
data_set['dob_years'].value_counts().sort_index()

19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    614
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

<b> 2.2.4. education

In [15]:
#выведем состав значений колонки образование
data_set['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

по колонке education кроме регистра проблем не установлено.

<b>2.2.5. education_id

In [16]:
data_set['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

По колонке Код вида образования (education_id) проблем не установлено

<b>2.2.6. family_status

In [17]:
data_set['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

По колонке family_status проблем не установлено

<b>2.2.7. family_status_id

In [18]:
data_set['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

По колонке family_status_id проблем не установлено

<b>2.2.8. gender

In [19]:
data_set['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [20]:
# заменим единичный случай значения 'XNA', поскольку такого определения пола не существует, 
# на "F" как наиболее часто стречающийся в таблице
data_set.loc[data_set['gender']=='XNA','gender'] = 'F'
data_set['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

По колонке gender других проблем не установлено.

<b>2.2.9. income_type

In [21]:
data_set['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

По колонке income_type проблем не установлено

<b>2.2.10. debt

In [22]:
data_set['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

По колонке debt проблем не установлено

<b>2.2.11. total_income

In [23]:
(data_set['total_income']//10000).describe()

count    21525.000000
mean        16.038467
std          9.807547
min          2.000000
25%         10.000000
50%         14.000000
75%         19.000000
max        226.000000
Name: total_income, dtype: float64

По колонке total_income проблем не установлено

<b>2.2.12. purpose

In [24]:
data_set['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Нечитаемых и случайных значений сведений о цели кредита не установлено. 
Данные о цели кредита заведены не стандартизовано, фактчески совпадающие цели кредита указываются по разному.

### Шаг 2.3. Изменение типов данных.

In [25]:
#приведем данные о доходе к целому типу
try:
    data_set['total_income'] = data_set['total_income'].astype('int')
except:
    print('Данные о доходах содержат нечисловые значения, требуется чекап этапа подготовки данных (битые данные, заполнение пропусков и т.п.)')

### Шаг 2.4. Удаление дубликатов.

Поскольку клиенты в исходных данных не идентифицированы - дедубликация повторяющихся строк нецелесообразна. Повторяющиеся строки могут быть как дублированной информацией, так и информацией о клиентах со сходными параметрами (доход, возраст и т.п.)
При этом имеют место:
- отмеченная ранее неявная дубликация по цели кредита, требующая категоризации (данная процедура описана далее по проекту).
- дубликация данных об образовании заемщиков, вызванная заведением названия уровня образования с разным регистром:

In [26]:
#приведем к нижнему регистру сведения об образовании
data_set['education'] = data_set['education'].str.lower()
data_set['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [27]:
data_set

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,NaN,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,NaN,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создаем словари education_dict и family_status_dict для обособления строковых значений по образованию и семейному статусу из основной таблицы. Это позволит разгрузить основную таблицу - в ней останутся только ID значений. 
ID будут отсылать к полному наименованию категории во вновь созданном справочнике.
Из исходной таблицы стобцы с полными наименованиями удаляем.

In [28]:
try:
    education_dict = data_set.loc[:,['education_id','education']]
    data_set = data_set.drop('education',axis=1)
except:
    print('Столбец education уже удалили')
education_dict = education_dict.drop_duplicates()

In [29]:
education_dict = education_dict.set_index('education_id').T.to_dict('list')
education_dict

{0: ['высшее'],
 1: ['среднее'],
 2: ['неоконченное высшее'],
 3: ['начальное'],
 4: ['ученая степень']}

In [30]:
try:
    family_status_dict = data_set.loc[:,['family_status_id','family_status']]
    data_set = data_set.drop('family_status',axis=1)
except:
    print('Столбец family_status уже удалили')
family_status_data = family_status_dict.drop_duplicates()
family_status_dict = family_status_dict.set_index('family_status_id').T.to_dict('list')
family_status_dict

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  import sys


{0: ['женат / замужем'],
 1: ['гражданский брак'],
 2: ['вдовец / вдова'],
 3: ['в разводе'],
 4: ['Не женат / не замужем']}

## Шаг 2.6. Категоризация дохода.

In [31]:
#объявляем функцию которая будет раскладывать на категории значение доходов клиентов
def func_income_category(total_income):
    try:
        if total_income <= 30000:
            return 'E'
        elif 30000<total_income<=50000:
            return 'D'
        elif 50000<total_income<=200000:
            return 'C'
        elif 200000<total_income<=1000000:
            return 'B'
        elif 1000000<total_income:
            return 'A'
    except:
        print('Данные о доходах содержат нечисловые значения, требуется чекап этапа подготовки данных (битые данные, заполнение пропусков и т.п.)')
        print('Значение вызвавшее ошибку',total_income)
        return ''


Обрабатываем столбец с доходами и заполняем столбец с категориями дохода клиента.

In [32]:
data_set['total_income_category'] = data_set['total_income'].apply(func_income_category)
data_set['total_income_category'].value_counts()

C    16086
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

В связи с тем что цель кредита заполнялась нестандартизованно, обработка и группировка данных по признаку purpose/цель кредита невозможна.

Осуществим категоризацию значений 'purpose' по следующим категориям:
'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.

In [33]:
# понижаю регистр в значениях в столбце purpose
data_set['purpose'] = data_set['purpose'].str.lower()
# возвращаю формулой категорию цели креда, определяя цель поиском ключа по цели кредита
def purpose_category (text_purpose):
    try:
        if ('жиль' in text_purpose) or ('недвиж' in text_purpose):
            return 'операции с недвижимостью'
        elif 'автомоб' in text_purpose:
            return 'операции с автомобилем'
        elif 'свадь' in text_purpose:
            return 'проведение свадьбы'
        elif 'образов' in text_purpose:
            return 'получение образования'
        else:
            print('Неклассифицируемое значение', text_purpose)
            return ''
    except:
        print("ошибка значения", text_purpose)
        return ''
#применяю функцию заполняющую стобец с категорией в дата сэте
data_set['purpose_category'] = data_set['purpose'].apply(purpose_category)
data_set['purpose_category'].value_counts()

операции с недвижимостью    10840
операции с автомобилем       4315
получение образования        4022
проведение свадьбы           2348
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [34]:
#создаю аггрегированную по значению "количество детей" таблицу
data_set_grouped=data_set.groupby('children').agg({'debt':['count','sum']})
#добавляю рассчетные значения "доля клиентов группы от всех клиентов", "среднее значение/вероятность просрочки по группе", "вероятность погашения = (1-вероятность просрочи)"
data_set_grouped=data_set_grouped.assign(group_share_in_total = data_set_grouped['debt']['count']/data_set_grouped['debt']['count'].sum(),\
       debt_children_rate=data_set_grouped['debt']['sum']/data_set_grouped['debt']['count'],\
       repaid_rate = 1-data_set_grouped['debt']['sum']/data_set_grouped['debt']['count'])
data_set_grouped

debt       group_share_in_total debt_children_rate repaid_rate
          count   sum                                                    
children                                                                 
0         14149  1063             0.657329           0.075129    0.924871
1          4865   445             0.226016           0.091470    0.908530
2          2131   202             0.099001           0.094791    0.905209
3           330    27             0.015331           0.081818    0.918182
4            41     4             0.001905           0.097561    0.902439
5             9     0             0.000418           0.000000    1.000000

##### Вывод 1:

Только заемщики у которых 5 детей не допускали случаев просроченной задолженности. Однако количество таких заемщиков ничтожно в общем объеме (менее 0,1%), представлены такие заемщики в банке вероятно нерепрезентативно относительно общей массы потенциальных клиентов с 5 детьми.
Заемщики не имеющие детей имеют наименьшую долю случаев несвоевременного погашения кредитов. 
При этом заемщики не имеющие детей, а также имеющие одного или двух детей составляют 98% клиентов (66%, 22%, 10% соответственно).
Статистика по данным трем группам свидетельствует об обратной корелляции вероятности погашения кредита и количества детей у заемщика - вероятность погашения уменьшается с 92,5 % для заемщиков без детей до 90,8 % и 90,5% у заемщиков с 2 и 3 детьми соответственно.


## Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
#объявляю функцию которая возвращает столбец с названиями категорий клиентов, соответствующих индексу
def status_from_dict(index_id):
    new_index = []
    for value in index_id:
        new_index.append(family_status_dict[value])
    return new_index
#создаю сводную таблицу и присоединаю к ней стобцы с расчетными значениями
family_pivot = data_set.pivot_table(index='family_status_id',values='debt', aggfunc=['sum','count','mean'])
family_pivot = family_pivot.assign(group_share_in_total = family_pivot['count']/family_pivot['count'].sum(),\
                                  repaid_rate = 1-family_pivot['mean'])
#добавляем названия категорий
family_pivot['family_status'] = status_from_dict(family_pivot.index)
#сортируем по вероятности погашения
family_pivot = family_pivot.sort_values(by=('repaid_rate',''),ascending=False)
family_pivot = family_pivot.rename(columns={'debt':'','sum':'unpaid_qunatity','count':'total_quantity','mean':'unpaid_rate'})
family_pivot

,unpaid_qunatity,total_quantity,unpaid_rate,group_share_in_total,repaid_rate,family_status
,,,,,,
family_status_id,,,,,,
2,63,960,0.065625,0.044599,0.934375,[вдовец / вдова]
3,85,1195,0.071130,0.055517,0.928870,[в разводе]
0,931,12380,0.075202,0.575145,0.924798,[женат / замужем]
1,388,4177,0.092890,0.194053,0.907110,[гражданский брак]
4,274,2813,0.097405,0.130685,0.902595,[Не женат / не замужем]


##### Вывод 2:

Наибольшая вероятность погашения установлена по заемщикам с социальным статусов "водвец/вдова" и "в разводе" - 93,4%, 92,8% соответственно. Наименьшая вероятность погашения установлена по группам заемщиков с социальным статусом "гражданский брак" и "не женат/не замужем". 
В связи с тем что по своей сути "гражданский брак" ближе остальных социальных статусов к статусу "не женат/не замужем" можно сделать вывод о том что заемщики не состоящие и не состоявшие в официальном браке чаще нарушают сроки погашения по кредитам.


## Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
#создаю словарь с названиями интервалов/категорий клиентов по доходам
income_dict = {'A':'      0 -  30 000',
              'B' :' 30 001 -  50 000',
              'C' :' 50 001 - 200 000',
              'D' :'200 001 -1000 000',
              'E': '1 000 001 ------->'}
#объявляю функцию которая возвращает столбец с названиями категорий клиентов, соответствующих переданному индексу
def income_from_dict(index_id):
    new_index = []
    for value in index_id:
        new_index.append(income_dict[value])
    return new_index
#создаю сводную таблицу и присоединаю к ней стобцы с расчетными значениями
income_pivot = data_set.pivot_table(index='total_income_category',values='debt', aggfunc=['sum','count','mean'])
income_pivot = income_pivot.assign(group_share_in_total = income_pivot['count']/income_pivot['count'].sum(),\
                                  repaid_rate = 1-income_pivot['mean'])
#добавляем названия категорий
income_pivot['family_status'] = income_from_dict(income_pivot.index)
#сортируем по вероятности погашения
income_pivot = income_pivot.sort_values(by=('repaid_rate',''),ascending=False)
#переименовываем столбцы
income_pivot = income_pivot.rename(columns={'debt':'','sum':'unpaid_qunatity','count':'total_quantity','mean':'unpaid_rate'})
income_pivot

,unpaid_qunatity,total_quantity,unpaid_rate,group_share_in_total,repaid_rate,family_status
,,,,,,
total_income_category,,,,,,
D,21,350,0.060000,0.016260,0.940000,200 001 -1000 000
B,356,5042,0.070607,0.234239,0.929393,30 001 - 50 000
A,2,25,0.080000,0.001161,0.920000,0 - 30 000
C,1360,16086,0.084546,0.747317,0.915454,50 001 - 200 000
E,2,22,0.090909,0.001022,0.909091,1 000 001 ------->


##### Вывод 3:

Наибольшая вероятность погашения приходится на группу клиентов с доходами '30 001-50 000' и '200 001-1 000 000' или 94% и 92,9%.
При этом промежуточная по сумме доходов группа клиентов с доходами "50 001-200 000" показывает вероятность погашения ниже - на уровне 91,5%
Оставшиеся две группы клиентов с доходами "0-30 000" и "1 000 001 и более" составляют 0,1% от общего числа клиентов (каждая) и не являются показательными ввиду недостаточного объема выборки.
Таким образом взаимосвязь между доходами клиентов не подтверждена.  

## Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [37]:
#создаю сводную таблицу и присоединаю к ней стобцы с расчетными значениями
purpose_pivot = data_set.pivot_table(index='purpose_category',values='debt', aggfunc=['sum','count','mean'])
purpose_pivot = purpose_pivot.assign(group_share_in_total = purpose_pivot['count']/purpose_pivot['count'].sum(),\
                                  repaid_rate = 1-purpose_pivot['mean'])
#сортируем по вероятности погашения
purpose_pivot = purpose_pivot.sort_values(by=('repaid_rate',''),ascending=False)
#переименовываем столбцы
purpose_pivot = purpose_pivot.rename(columns={'debt':'','sum':'unpaid_qunatity','count':'total_quantity','mean':'unpaid_rate'})
purpose_pivot

,unpaid_qunatity,total_quantity,unpaid_rate,group_share_in_total,repaid_rate
,,,,,
purpose_category,,,,,
операции с недвижимостью,782,10840,0.072140,0.503600,0.927860
проведение свадьбы,186,2348,0.079216,0.109082,0.920784
получение образования,370,4022,0.091994,0.186852,0.908006
операции с автомобилем,403,4315,0.093395,0.200465,0.906605


##### Вывод 4:

По результатам анализа установлено, что своевременность возврата кредита выше у кредитов на покупку или иные операции с недвижимостью, а также на проведение свадьбы 92,7% и 92,1%
Наихудшее качество погашения кредитов установлено по кредитам на образование и приобретение автотранспорта 90,8% и 90,6% соответственно.

## Общий вывод:

Для анализа представлены данные по 21,5 тыс. клиентов - заемщиков разного возраста/дохода и целей кредита.
Порядка 10% записей не содержали пропуски в сведениях о доходе и стаже. Общие признаки у таких заемщиков отсутствовали. 
Предположительно эта часть данных получена из канала продаж в котором такая информация не анализировалась 
при рассмотрении вопроса о выдаче кредита, либо по какой то причине не сохранялась. 
Например если данные заводились в партнерской CRM.

Необходимо отметить системную проблему некорректных сведений о рабочем стаже по клиентам-пенсионерам. 
Целесообразно проследить путь который проходят эти данные от клиентского работника до выгрузки, представленной на анализ.

Анализ данных сформированных по результатам предобработки подтвердил следующие гипотезы
    1. Верояность погашения связана с наличием и количеством детей у заемщика - с ростом количества детей 
    снижается вероятность своевременного погашения кредита.
    2. Вероятность погашения ниже для клиентов ранее не обременявших себя официальным браком. Что может свидетельствовать 
    как об ответственности так в равной степени и о наличии привычки планирования бюджета у людей состоявших в официальном браке.
    3. Вероятность погашения не связана с уровнем дохода заемщиков. При этом информация о долговой нагрузке 
    менее обеспеченных заемщиков из датасета отсутствует. Возможно при предоставлении кредитов учитывалась способность погашения
    кредитов что, как следствие выровняло вероятность погашения в разных группах по уровню дохода.
    4. Веротность погашения кредитов, использванных на цели приобретения недвижимости и проведение свадьбы выше, чем по
    образовательным кредитам и автокредитам.